### Imports

In [171]:
import os
import numpy as np
import pandas as pd

### Inputs

In [172]:
radnet_input_df = pd.read_csv("../inputs/radnet_input.csv")
radnet_input_apmr_edge = radnet_input_df['apmr_edge'].values
radnet_input_mfsr_edge = radnet_input_df['mfsr_edge'].values
radnet_input_idg_global = radnet_input_df['idg_global'].values

- apmr_edge (Active Prefix Message Ratio): Taxa de eficiência dos Active Prefixes no encaminhamento de mensagens através das arestas, 
medindo quão bem os campos de prefixo conseguem realizar matching probabilístico para encaminhar mensagens entre nós vizinhos.

- mfsr_edge (Message Forwarding Success Ratio): Proporção de mensagens transmitidas com sucesso através de uma aresta específica, 
considerando interferência, congestionamento e qualidade da conexão sem fio entre dois nós conectados.

- idg_global (Interest Group Delivery): Eficácia global da entrega de mensagens no modo Interest-Group addressing, 
medindo quão bem o protocolo consegue localizar e entregar mensagens aos nós que compartilham o mesmo interesse da aplicação.

In [173]:
inference_output_df = pd.read_csv("../outputs/inference_output.csv")
inference_output_class = inference_output_df['class'].values
inference_output_confidence = inference_output_df['confidence'].values

- confidence (Confiança da Inferência do Modelo GNN): Eficácia do modelo gerado pelo GNN medido na atapa anterior.

In [174]:
edges_input_df = pd.read_csv("../inputs/edges_input.csv")
edges_input = edges_input_df.values.tolist()

### Fusion

In [ ]:
# pesos podenrados
w_model, w_apmr, w_mfsr, w_idg = 0.4, 0.25, 0.25, 0.1
w_sum = w_model + w_apmr + w_mfsr + w_idg

In [ ]:
confidence_model = np.where(inference_output_class == 1,
    inference_output_confidence,
    1.0 - inference_output_confidence
)

In [ ]:
fused = (
    w_model * confidence_model +
    w_apmr * radnet_input_apmr_edge +
    w_mfsr * radnet_input_mfsr_edge +
    w_idg  * radnet_input_idg_global
) / w_sum

In [178]:
# classe final a partir do filtrado
cls = (fused >= 0.5).astype(int)

# confiança orientada à classe escolhida
conf = np.where(cls == 1, fused, 1.0 - fused)

### Apply

In [179]:
print(f"{'RSSI':<6} {'ETX':<5} {'Delay':<6} {'Busy':<6} {'Classe':<8} {'Status':<8} {'Confiança':<10}")
print(f"{'-'*55}")

CLASS_NAMES = {0: "Ruim", 1: "Bom"}
for i, row in enumerate(edges_input):
    print(f"{row[0]:<6.0f} {row[1]:<5.1f} {row[2]:<6.0f} {row[3]:<6.2f} "
          f"{cls[i]:<8} {CLASS_NAMES[int(cls[i])]:<8} {conf[i]:<10.2f}")

RSSI   ETX   Delay  Busy   Classe   Status   Confiança 
-------------------------------------------------------
-89    4.9   47     0.43   0        Ruim     0.52      
-87    8.3   74     0.44   0        Ruim     0.52      
-82    6.9   98     0.34   1        Bom      0.50      
-70    9.6   76     0.93   1        Bom      0.52      
-62    8.7   61     0.82   1        Bom      0.53      
-73    8.9   36     0.91   1        Bom      0.52      
-80    4.7   62     0.74   1        Bom      0.55      
-60    6.6   100    0.47   0        Ruim     0.52      
-77    9.9   78     0.43   1        Bom      0.56      
-89    9.5   78     0.86   1        Bom      0.53      
-95    8.3   98     0.57   0        Ruim     0.51      
-72    9.5   68     0.52   1        Bom      0.50      
-90    7.5   48     0.90   1        Bom      0.53      
-88    5.1   76     0.97   0        Ruim     0.50      
-85    8.9   79     0.87   1        Bom      0.53      
-78    4.3   81     0.33   1        Bom      0.5

### Save

In [180]:
fusion_output = pd.DataFrame({
    'class': cls,
    'confidence': conf
})

os.makedirs("../outputs", exist_ok=True)
fusion_output.to_csv("../outputs/fusion_output.csv", index=False)

print("Fusão de Dados salva em ../outputs/fusion_output.csv")

Fusão de Dados salva em ../outputs/fusion_output.csv
